# Model selection

Initialisation
- If reproducibility of results is needed, set the seed of the random number generator as needed

In [56]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Data onboarding for loading, understanding, transforming and splitting for supervised learning
- Load the data
- Find out how the data oriented and determine the sizes
- Visualise the data
- Transform the data set so that it is suitable for training a neural network

In [57]:
df = pd.read_csv("humanactivity.csv")

# Creating the data
X = df.iloc[:,1:].values

num_features = X.shape[1]
print("Number of features:", num_features)
print("Global min:", X.min())
print("Global max:", X.max())


y = df.iloc[:,0].values.astype(int) -1

num_classes = len(np.unique(y))
print("Number of classes:", num_classes)

unique, counts = np.unique(y, return_counts=True)
class_counts = dict(zip(unique, counts))
print("Samples per class:")
for cls, cnt in class_counts.items():
    print(f"Class {cls}: {cnt} samples")

# train/test split
split = int(0.8 * X.shape[0])
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

Number of features: 60
Global min: -5.28682646609656
Global max: 1969.19770431434
Number of classes: 5
Samples per class:
Class 0: 5850 samples
Class 1: 6220 samples
Class 2: 5396 samples
Class 3: 3956 samples
Class 4: 2653 samples


Neural network architecture
- Design a neural network for solving the classification task

In [58]:
import numpy as np

def softmax(x):
    e = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e / e.sum(axis=1, keepdims=True)

def cross_entropy(pred, target):
    # target: one-hot
    return -np.mean(np.sum(target * np.log(pred + 1e-12), axis=1))

class RNNClassifier:
    def __init__(self, input_size, hidden_size, output_size, lr=1e-3, seed=0):
        rng = np.random.RandomState(seed)

        self.Wxh = rng.randn(hidden_size, input_size) / np.sqrt(input_size)
        self.Whh = rng.randn(hidden_size, hidden_size) / np.sqrt(hidden_size)
        self.Why = rng.randn(output_size, hidden_size) / np.sqrt(hidden_size)

        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

        self.hidden_size = hidden_size
        self.lr = lr

    def forward(self, X):
        batch, seq_len, _ = X.shape
        h = np.zeros((batch, seq_len + 1, self.hidden_size))

        for t in range(seq_len):
            xt = X[:, t, :]
            pre = xt @ self.Wxh.T + h[:, t, :] @ self.Whh.T + self.bh.T
            h[:, t+1, :] = np.tanh(pre)

        logits = h[:, -1, :] @ self.Why.T + self.by.T
        y_pred = softmax(logits)

        return h, y_pred

    def backward(self, X, h, y_pred, y_true):
        batch, seq_len, _ = X.shape

        dWxh = np.zeros_like(self.Wxh)
        dWhh = np.zeros_like(self.Whh)
        dWhy = np.zeros_like(self.Why)
        dbh = np.zeros_like(self.bh)
        dby = np.zeros_like(self.by)

        dy = (y_pred - y_true) / batch
        h_last = h[:, -1, :]

        dWhy += dy.T @ h_last
        dby += dy.sum(axis=0, keepdims=True).T

        dh_next = dy @ self.Why

        for t in reversed(range(seq_len)):
            ht = h[:, t+1, :]
            ht_prev = h[:, t, :]

            dt = dh_next * (1 - ht**2)  # tanh'

            dbh += dt.sum(axis=0)[:, None]
            dWxh += dt.T @ X[:, t, :]
            dWhh += dt.T @ ht_prev

            dh_next = dt @ self.Whh

        # clip
        for g in [dWxh, dWhh, dWhy, dbh, dby]:
            np.clip(g, -5, 5, out=g)

        # update
        self.Wxh -= self.lr * dWxh
        self.Whh -= self.lr * dWhh
        self.Why -= self.lr * dWhy
        self.bh  -= self.lr * dbh
        self.by  -= self.lr * dby

    def train(self, X, y, epochs=100, batch_size=16):
        loss_history = []
        n = X.shape[0]
        for epoch in range(epochs):
            perm = np.random.permutation(n)
            epoch_loss = 0

            for i in range(0, n, batch_size):
                xb = X[perm[i:i+batch_size]]
                yb = y[perm[i:i+batch_size]]

                h, y_pred = self.forward(xb)
                loss = cross_entropy(y_pred, yb)
                epoch_loss += loss * xb.shape[0]

                self.backward(xb, h, y_pred, yb)
            
            epoch_loss /= n
            loss_history.append(epoch_loss)

            print(f"Epoch {epoch+1}: loss={epoch_loss:.4f}")
        return loss_history


Model training
- Train the selected model using the training data

In [59]:
# reshape for RNN
X_train = X_train.reshape(len(X_train), 1, -1)
X_test  = X_test.reshape(len(X_test), 1, -1)

num_classes = len(np.unique(y_train))
y_train_oh = np.eye(num_classes)[y_train]
y_test_oh  = np.eye(num_classes)[y_test]

rnn = RNNClassifier(input_size=X_train.shape[2],
                    hidden_size=256,
                    output_size=num_classes,
                    lr=1e-2)

losses = rnn.train(X_train, y_train_oh, epochs=100)

plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Curve")
plt.grid(True)
plt.show()


Epoch 1: loss=0.2242
Epoch 2: loss=0.1659
Epoch 3: loss=0.1579
Epoch 4: loss=0.1554
Epoch 5: loss=0.1502
Epoch 6: loss=0.1531
Epoch 7: loss=0.1421
Epoch 8: loss=0.1384
Epoch 9: loss=0.1434
Epoch 10: loss=0.1415
Epoch 11: loss=0.1422
Epoch 12: loss=0.1368
Epoch 13: loss=0.1476
Epoch 14: loss=0.1380
Epoch 15: loss=0.1334
Epoch 16: loss=0.1340
Epoch 17: loss=0.1347
Epoch 18: loss=0.1321
Epoch 19: loss=0.1244
Epoch 20: loss=0.1316
Epoch 21: loss=0.1276
Epoch 22: loss=0.1302
Epoch 23: loss=0.1287
Epoch 24: loss=0.1250
Epoch 25: loss=0.1270
Epoch 26: loss=0.1306
Epoch 27: loss=0.1306
Epoch 28: loss=0.1310
Epoch 29: loss=0.1299
Epoch 30: loss=0.1299
Epoch 31: loss=0.1207
Epoch 32: loss=0.1172
Epoch 33: loss=0.1261
Epoch 34: loss=0.1224
Epoch 35: loss=0.1205
Epoch 36: loss=0.1195
Epoch 37: loss=0.1201
Epoch 38: loss=0.1215
Epoch 39: loss=0.1225
Epoch 40: loss=0.1200
Epoch 41: loss=0.1191
Epoch 42: loss=0.1211
Epoch 43: loss=0.1184
Epoch 44: loss=0.1236
Epoch 45: loss=0.1199
Epoch 46: loss=0.12

KeyboardInterrupt: 

Model testing
- Classify the test set samples by applying the trained/optimised model
- Analyse the experimental results

In [ ]:
import matplotlib.pyplot as plt

# Predict
_, y_pred_test = rnn.forward(X_test)

# Class label predictions
pred_classes = np.argmax(y_pred_test, axis=1)
true_classes = np.argmax(y_test_oh, axis=1)

accuracy = np.mean(pred_classes == true_classes)
print("Test accuracy:", accuracy)

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(true_classes, pred_classes)
print("Confusion Matrix:\n", cm)

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.title("Confusion Matrix")
plt.show()

Test accuracy: 0.9607476635514018
